Import des libs & enregistrement du dataset COCO

Enregistrer ton dataset

In [3]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances  # ← assure-toi que cette ligne est bien là

import os

# Chemin vers ton dataset
dataset_path = "image annotation cv_collab.v3i.coco"

# Enregistrement des datasets COCO (train + valid)
register_coco_instances(
    "web_train", {}, 
    os.path.join(dataset_path, "train", "_annotations.coco.json"),
    os.path.join(dataset_path, "train")
)

register_coco_instances(
    "web_valid", {}, 
    os.path.join(dataset_path, "valid", "_annotations.coco.json"),
    os.path.join(dataset_path, "valid")
)

# Attribution des classes
MetadataCatalog.get("web_train").thing_classes = ['content', 'advertisement', 'footer', 'header', 'left sidebar', 'logo', 'media', 'pop up', 'right side bar']
MetadataCatalog.get("web_valid").thing_classes = ['content', 'advertisement', 'footer', 'header', 'left sidebar', 'logo', 'media', 'pop up', 'right side bar']


Configurer le modèle Faster R-CNN

In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml"))

cfg.DATASETS.TRAIN = ("web_train",)
cfg.DATASETS.TEST = ("web_valid",)
cfg.DATALOADER.NUM_WORKERS = 2

# Utilise un modèle pré-entraîné
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml")

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # learning rate
cfg.SOLVER.MAX_ITER = 1000     # ajustable (300 suffisant pour 11 images)
cfg.SOLVER.STEPS = []

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9  # nombre de classes

cfg.OUTPUT_DIR = "./output_detectron_web"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
cfg.MODEL.DEVICE = "cpu"
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader

# evaluator = COCOEvaluator("web_valid", cfg, False, output_dir=cfg.OUTPUT_DIR)
# val_loader = build_detection_test_loader(cfg, "web_valid")

# print("🧪 Évaluation en cours...")
# inference_on_dataset(trainer.model, val_loader, evaluator)



[04/09 20:00:04 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

model_final_68d202.pkl: 663MB [25:54, 426kB/s]                                
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (10, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (10,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (36, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads

[04/09 20:26:00 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\DELL\anaconda3\envs\detectron2-env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/09 20:32:14 d2.utils.events]:  eta: 4:27:32  iter: 19  total_loss: 4.106  loss_cls: 2.223  loss_box_reg: 0.7293  loss_rpn_cls: 0.8696  loss_rpn_loc: 0.2047    time: 17.5281  last_time: 14.9907  data_time: 0.6975  last_data_time: 0.0066   lr: 4.9953e-06  
[04/09 20:37:44 d2.utils.events]:  eta: 4:03:03  iter: 39  total_loss: 3.678  loss_cls: 2.068  loss_box_reg: 0.7065  loss_rpn_cls: 0.7777  loss_rpn_loc: 0.1958    time: 16.8403  last_time: 22.7729  data_time: 0.0072  last_data_time: 0.0070   lr: 9.9902e-06  
[04/09 20:43:04 d2.utils.events]:  eta: 3:55:04  iter: 59  total_loss: 3.24  loss_cls: 1.851  loss_box_reg: 0.6818  loss_rpn_cls: 0.4919  loss_rpn_loc: 0.1863    time: 16.5463  last_time: 21.7572  data_time: 0.0067  last_data_time: 0.0064   lr: 1.4985e-05  
[04/09 20:49:03 d2.utils.events]:  eta: 3:56:35  iter: 79  total_loss: 2.766  loss_cls: 1.524  loss_box_reg: 0.7403  loss_rpn_cls: 0.3509  loss_rpn_loc: 0.1654    time: 16.8977  last_time: 15.5174  data_time: 0.0049  last_da

In [ ]:
# model = trainer.model
# print(model)
##couches modifiées
# print(model.roi_heads.box_predictor)
##poids appris
# # Poids de classification
# print(model.roi_heads.box_predictor.cls_score.weight)

# # Biais
# print(model.roi_heads.box_predictor.cls_score.bias)


In [6]:
import shutil
import torch
import pickle

# 1. Chemin du modèle final entraîné
trained_model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Vérifie que le modèle existe
if os.path.exists(trained_model_path):
    print(f"Modèle sauvegardé ici : {trained_model_path}")
else:
    print("Aucun modèle trouvé. Vérifie que l'entraînement est terminé avec succès.")

# 2. Sauvegarde de la configuration utilisée (fichier .pkl ou .yaml)
with open(os.path.join(cfg.OUTPUT_DIR, "config.pkl"), "wb") as f:
    pickle.dump(cfg, f)
    print("Configuration sauvegardée : config.pkl")

# 3. Sauvegarde des métadonnées (optionnel mais conseillé)
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
with open(os.path.join(cfg.OUTPUT_DIR, "metadata.pkl"), "wb") as f:
    pickle.dump(metadata, f)
    print("Metadata sauvegardées : metadata.pkl")


Modèle sauvegardé ici : ./output_detectron_web\model_final.pth
Configuration sauvegardée : config.pkl
Metadata sauvegardées : metadata.pkl


In [13]:
import os, random, cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo

# === Chargement du modèle ===
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml"))
cfg.OUTPUT_DIR = "./output_detectron_web"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)

# === Métadonnées ===
thing_classes = ['content', 'advertisement', 'footer', 'header', 'left sidebar', 'logo', 'media', 'pop up', 'right side bar']
MetadataCatalog.get("web_custom").thing_classes = thing_classes
metadata = MetadataCatalog.get("web_custom")

# === Couleurs RGB normalisées [0, 1] ===
CLASS_COLORS = {
    "content": (255, 0, 0),
    "advertisement": (0, 255, 0),
    "footer": (0, 0, 255),
    "header": (255, 255, 0),
    "left sidebar": (255, 0, 255),
    "logo": (0, 255, 255),
    "media": (128, 128, 0),
    "pop up": (0, 128, 255),
    "right side bar": (128, 0, 128),
}

# === Préparer les images ===
input_dir = "dataset_images"
output_dir = "predictions_vis_all"
os.makedirs(output_dir, exist_ok=True)

image_files = [f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
selected_images = random.sample(image_files, min(10, len(image_files)))

# === Annoter et enregistrer ===
for img_name in selected_images:
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    labels = instances.pred_classes
    boxes = instances.pred_boxes
    scores = instances.scores

    # Ajouter le score au label
    label_names = [f"{thing_classes[i]}: {scores[j]:.2f}" for j, i in enumerate(labels)]
    colors = [tuple(c / 255 for c in CLASS_COLORS[thing_classes[i]]) for i in labels]

    v = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.SEGMENTATION)
    v = v.overlay_instances(boxes=boxes, labels=label_names, assigned_colors=colors)
    result = v.get_image()[:, :, ::-1]

    # Enregistrement de l'image annotée
    save_path = os.path.join(output_dir, f"annotated_{img_name}")
    cv2.imwrite(save_path, result)
    print(f"✔ Image annotée sauvegardée : {save_path}")


[04/10 07:41:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_detectron_web\model_final.pth ...
✔ Image annotée sauvegardée : predictions_vis_all\annotated_articles_8_1743368487.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_blog_OR_politics_OR_news_12_1743280851.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_54_1743314761.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_health_OR_fitness_OR_wellness_52_1743419892.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_blog_OR_politics_OR_news_138_1743285615.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_blog_OR_politics_OR_news_84_1743414734.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_health_OR_fitness_OR_wellness_136_1743308641.jpg
✔ Image annotée sauvegardée : predictions_vis_all\annotated_AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_30_1

In [15]:
import os, random, cv2
import torch
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.structures import Boxes


# === Chargement du modèle ===
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml"))
cfg.OUTPUT_DIR = "./output_detectron_web"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

# === Métadonnées ===
thing_classes = ['content', 'advertisement', 'footer', 'header', 'left sidebar', 'logo', 'media', 'pop up', 'right side bar']
MetadataCatalog.get("web_custom").thing_classes = thing_classes
metadata = MetadataCatalog.get("web_custom")

# === Couleurs normalisées [0, 1] ===
CLASS_COLORS = {
    "content": (255, 0, 0),
    "advertisement": (0, 255, 0),
    "footer": (0, 0, 255),
    "header": (255, 255, 0),
    "left sidebar": (255, 0, 255),
    "logo": (0, 255, 255),
    "media": (128, 128, 0),
    "pop up": (0, 128, 255),
    "right side bar": (128, 0, 128),
}

# === Préparer les images ===
input_dir = "dataset_images"
output_dir = "predictions_vis_header_footer"
os.makedirs(output_dir, exist_ok=True)

image_files = [f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
selected_images = random.sample(image_files, min(10, len(image_files)))  # modifie si tu veux toutes

# === Annoter et enregistrer ===
for img_name in selected_images:
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)

    outputs = predictor(img)
    instances = outputs["instances"].to("cpu")
    labels = instances.pred_classes
    boxes = instances.pred_boxes
    scores = instances.scores

    selected_boxes = []
    selected_labels = []
    selected_colors = []

    for cls_id in set(labels.tolist()):
        cls_name = thing_classes[cls_id]
        indices = [i for i, c in enumerate(labels) if c == cls_id]

        if cls_name in ["header", "footer"]:
            if len(indices) > 1:
                # Choisir la box avec le score max
                best_idx = max(indices, key=lambda i: scores[i])
                score = scores[best_idx].item()
                label = f"{cls_name}: {score:.2f}"
                selected_boxes.append(boxes[best_idx])
                selected_labels.append(label)
                selected_colors.append(tuple(c / 255 for c in CLASS_COLORS[cls_name]))
            else:
                # Annoter normalement (seule box)
                for i in indices:
                    score = scores[i].item()
                    label = f"{cls_name}: {score:.2f}"
                    selected_boxes.append(boxes[i])
                    selected_labels.append(label)
                    selected_colors.append(tuple(c / 255 for c in CLASS_COLORS[cls_name]))
        else:
            # Toutes les boxes des autres classes
            for i in indices:
                score = scores[i].item()
                label = f"{cls_name}: {score:.2f}"
                selected_boxes.append(boxes[i])
                selected_labels.append(label)
                selected_colors.append(tuple(c / 255 for c in CLASS_COLORS[cls_name]))

    # === Visualisation ===
    v = Visualizer(img[:, :, ::-1], metadata=metadata, instance_mode=ColorMode.SEGMENTATION)
    if selected_boxes:
        if selected_boxes:
            box_tensors = torch.stack([b.tensor[0] for b in selected_boxes])  # [N, 4]
            selected_boxes = Boxes(box_tensors)
        v = v.overlay_instances(
            boxes=selected_boxes,
            labels=selected_labels,
            assigned_colors=selected_colors
        )
        result = v.get_image()[:, :, ::-1]
    else:
        result = img

    save_path = os.path.join(output_dir, f"annotated_{img_name}")
    cv2.imwrite(save_path, result)
    print(f"✔ Image sauvegardée : {save_path}")


[04/10 07:52:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_detectron_web\model_final.pth ...
✔ Image sauvegardée : predictions_vis_header_footer\annotated_Food_OR_Cooking_OR_Recipes_9_1743416104.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_articles_23_1743369032.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_101_1743315378.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_AI_OR_Data_Science_OR_Machine_Learning_OR_Deep_Learning_144_1743316108.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_education__116_1743292500.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_health_OR_fitness_OR_wellness_35_1743095662.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_health_OR_fitness_OR_wellness_78_1743435351.jpg
✔ Image sauvegardée : predictions_vis_header_footer\annotated_fashion_OR_clothing_OR_s